from https://arxiv.org/pdf/2106.11202.pdf, Table 6

In [ ]:
import pysm3.units as u

In [ ]:
fwhm = [1.57, 1.17, 1.04] * u.arcmin

In [ ]:
freqs = [93.8, 147.0, 219.9] * u.GHz

In [ ]:
bandwidth = [26.4, 32.5, 53.6] * u.GHz

In [ ]:
import numpy as np

In [ ]:
from collections import OrderedDict
from astropy import units as u
from astropy.table import QTable

In [ ]:
ipac_warning = [
    "Text file in IPAC table format, read with astropy",
    "from astropy.table import QTable",
    "QTable.read('filename.tbl', format='ascii.ipac')",
    f"Instrument model exported from `Arxiv 2106.11202` and from files by Tom Crawford",
]

In [ ]:
bands = ["95ghz", "150ghz", "220ghz"]

In [ ]:
table = QTable(
    data=[bands, freqs, fwhm, bandwidth],
    names=["band", "center_frequency", "fwhm", "bandwidth"],
)

In [ ]:
for key in ["bandpass", "beam"]:
    table[f"{key}_file"] = [f"{key}_{tag}.tbl" for tag in bands]

In [ ]:
table["nside"] = 0
table

In [ ]:
table.add_index("band")

In [ ]:
cd ../../private/202308_panexp_spt3g

In [ ]:
table.meta["comments"] = ipac_warning

In [ ]:
table.write(
    "spt_instrument_model.tbl",
    format="ascii.ipac",
    overwrite=True,
)

## Bandpasses

In [ ]:
%cd ../../private/202308_panexp_spt3g/

In [ ]:
bandpass = {}
for freq in bands:
    bandpass[freq] = QTable.read(f"bandpass_{freq.lower()}.txt", format="ascii")
    bandpass[freq].rename_column("freq", "bandpass_frequency")
    bandpass[freq]["bandpass_frequency"].unit = u.GHz
    bandpass[freq].rename_column("spectrum", "bandpass_weight")
    bandpass[freq].meta["comments"] = ipac_warning
    bandpass[freq].write(f"bandpass_{freq}.tbl", format="ascii.ipac", overwrite=True)

## Beams

In [ ]:
beams = QTable.read("compiled_2020_beams.txt", format="ascii", names=["ell"]+bands)

In [ ]:
beam = {}
for freq in bands:
    beam[freq] = QTable(data=[beams["ell"], beams[str(freq)]], names=["ell", "B"])
    beam[freq]["B"] /= np.max(beam[freq]["B"]) 
    beam[freq].meta["comments"] = ipac_warning
    beam[freq].write(f"beam_{freq}.tbl", format="ascii.ipac", overwrite=True)